# **Hazlan's Thesis**

## Judul: Analisis Deteksi Bahasa Isyarat Indonesia (BISINDO) Dan Sistem Isyarat Bahasa Indonesia (SIBI) Secara Real-Time Menggunakan Mediapipe dan Bidirectional Gated Recrurrent Unit (BiGRU)

## Dosen Pembimbing: Budi Santosa S.Si M.Sc

#### Change Log V4:

##### - 3 Kata -> 10 Kata
##### - Data Preprocessing changed (Split first, then augment)
##### - Modify Detection Interface

# 1. Import Library

In [ ]:
# Math and Data Processing
import os
import numpy as np
from matplotlib import pyplot as plt
import time
import shutil

# Sklearn
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score, classification_report

# Tensor & Keras
from tensorflow.keras.layers import Dense, GRU, Bidirectional, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential, load_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

# Evaluation
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

# Record and Testing Realtime
from scipy import stats
import mediapipe as mp
import cv2

# 2. Keypoints using MP Holistic

In [ ]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

# ==

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

# ==

def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))

# 3. Extract Keypoint Values

In [ ]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

# 4. Setup Folders for Collection

In [ ]:
# Path for exported data, numpy arrays
# Utility untuk Pembuatan Folder automatis untuk penyimpaman data koordinat lokasi pose, wajah, dan gerakan tangan -> .npy format
# BISINDO
# Paths for original and augmented data
DATA_PATH = os.path.join('/Users/hzlnqodrey/Developer/SKRIPSI/SIDANG/DATASET/BISINDO_DATASET_SIDANG_26Sep2024')
AUGMENTED_DATA_PATH = os.path.join('/Users/hzlnqodrey/Developer/SKRIPSI/SIDANG/DATASET/BISINDO_DATASET_AUGMENTED')

# # Remove the existing augmented data dir
# if os.path.exists(AUGMENTED_DATA_PATH):
#     shutil.rmtree(AUGMENTED_DATA_PATH)

# # Delete the directory and all its contents
# # Create the new augmented data dir
# os.makedirs(AUGMENTED_DATA_PATH)

# Total Data (BEFORE AUGMENT) = 
# 30 x 10 = 300 data original

# Total Data (AFTER AUGMENT) = 
# Training ((27x12) + 27 ) x 10  = 3510 data
# Testing 3 x 10 = 30 data 

In [ ]:
# Actions that we try to detect
actions = np.array(['Hai', 'Aku', 'Cinta', 'Kamu', 'Teman', 'Makan', 'Hari Ini', 'Bantu', 'Terima Kasih', 'Maaf'])

# Number of sequences and sequence length
no_sequences = 30
sequence_length = 30
no_sequences_augmented = 12  # Number of augmentations per sequence

In [ ]:
# RUN 1x UNTUK BUAT FOLDER SAJA (JANGAN DI RUN LAGI) UNTUK RECORDING DATA 
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing

In [ ]:
# RUN 1x UNTUK BUAT FOLDER SAJA (JANGAN DI RUN LAGI) UNTUK RECORDING DATA 
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(0, no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):
                
                # Read feed
                ret, frame = cap.read()

                # Save Extracted Landmarks to .npy file
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0,255, 0), 2, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,75), cv2.FONT_HERSHEY_SIMPLEX, 1.7, (0, 0, 255), 2, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('Hazlan BISINDO Record Webcam', image)
                    cv2.waitKey(3000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,75), cv2.FONT_HERSHEY_SIMPLEX, 1.7, (0, 0, 255), 2, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('Hazlan BISINDO Record Webcam', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                # npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                cv2.imwrite(npy_path + '.jpg', frame)
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

# 6. Split Data [Preprocess] 

####  - TO GET TEST SET  
####  - Partition Folder 
####  - Augment Folder Prep 

In [ ]:
# == Label map for the actions
label_map = {label:num for num, label in enumerate(actions)}

# == Collect all sequence numbers for each action
sequence_numbers = list(range(no_sequences))  # [0, 1, 2, ..., 29]
print(f"Sequence Numbers: {sequence_numbers}")


# == Split the sequence numbers into training and test sets (90% train, 10% test)
train_sequences, test_sequences = train_test_split(sequence_numbers, test_size=0.1)
print(f"Train sequence numbers: {train_sequences}")
print(f"Test sequence numbers: {test_sequences}")


# == Initialize lists to store training and test data
X_train, y_train, X_test, y_test = [], [], [], []

## == 6.1 Partition Folder Splitting

In [ ]:
# Load original sequences into X_train and X_test
for action in actions:
    # label = label_map[action]  # Get the label for this action
    
    for sequence in range(no_sequences):  # Loop through each sequence (folder)
        window = []  # Store the frames for this sequence
        for frame_num in range(sequence_length):
            # Load the frame from the sequence folder
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        
        # Partition data based on whether the sequence is in train or test
        if sequence in train_sequences:
            X_train.append(window)
            y_train.append(label_map[action])
        elif sequence in test_sequences:
            X_test.append(window)
            y_test.append(label_map[action])

# ==

y_train = to_categorical(y_train).astype(int)
y_test = to_categorical(y_test).astype(int)

# == Convert to numpy arrays
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Print shapes to confirm
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# == 6.2 AUGMENTATION FOLDER PREP

In [ ]:
#  Sort the train sequences
train_sequence_numbers = sorted(train_sequences)  # Sort the train sequences
print(f"Sorted Train Sequence Numbers: {train_sequence_numbers}")

# Create a mapping for original sequences to new indices
sequence_mapping = {}
# Map sorted train sequences to [0 ... 26]
for idx, seq_num in enumerate(train_sequence_numbers):
    sequence_mapping[seq_num] = idx

# Step 3: Generate augmented sequence numbers starting from 27
print(no_sequences_augmented)
print(len(train_sequences))

start_augmented_sequence = 27
end_augmented_sequence = 350
augmented_sequence_numbers = list(range(start_augmented_sequence, end_augmented_sequence + 1))

print(end_augmented_sequence)
print(len(augmented_sequence_numbers))

# Print augmented sequence numbers for debugging
print(f"Augmented sequence numbers: {augmented_sequence_numbers}")

# Combine original train sequences and augmented sequences
combined_sequences = sorted(train_sequence_numbers + augmented_sequence_numbers)
print(f"Combined sequence numbers (train [just showing, the new sorted is in sequence_mapping] + augmented): {combined_sequences}")

import shutil

# Create folder structure for combined data
for action in actions:
    # Create folders for training data
    for seq_num, new_idx in sequence_mapping.items():
        folder_path = os.path.join(AUGMENTED_DATA_PATH, action, str(new_idx))
        print(f"Creating folder for training data: {folder_path}")
        try:
            os.makedirs(folder_path)  # Create the folder for the new index
        except FileExistsError:
            pass  # If the folder exists, move on

        # Copy original files to the new folder
        for frame_num in range(sequence_length):
            original_file_path = os.path.join(DATA_PATH, action, str(seq_num), "{}.npy".format(frame_num))
            new_file_path = os.path.join(folder_path, "{}.npy".format(frame_num))
            shutil.copy(original_file_path, new_file_path)  # Copy the .npy file
            # print(f"Copied {original_file_path} to {new_file_path}")  # Print progress

    # Create folders for augmented data
    for aug_seq_num in augmented_sequence_numbers:
        folder_path = os.path.join(AUGMENTED_DATA_PATH, action, str(aug_seq_num))
        print(f"Creating folder for augmented data: {folder_path}")
        try:
            os.makedirs(folder_path)  # Create the folder for the augmented index
        except FileExistsError:
            pass  # If the folder exists, move on


# 7. Augment Data

In [ ]:
def augmentation_rotation(image, angle):
    # Rotate the image
    array = [15,30,345,330]
    image_list = []
    for element in array:
        rows, cols, _ = image.shape
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), element, 1)
        augmented_image = cv2.warpAffine(image, M, (cols, rows))
        image_list.append(augmented_image)
    # Show to screen
    return image_list

def zoomin(image, zoom_factor):
    height, width = image.shape[:2]
    new_width = int(width * zoom_factor)
    new_height = int(height * zoom_factor)
    left = int((width - new_width))
    top = int((height - new_height))
    right = int((width + new_width))
    bottom = int((height + new_height))
    cropped_image = image[top:bottom, left:right]
    zoom_in_width = int(image.shape[1])
    zoom_in_height = int(image.shape[0])
    zoom_in_image = cv2.resize(cropped_image, (zoom_in_width, zoom_in_height))
    return zoom_in_image

def augmentation_zoomin(image):
    array = [0.8,0.7]
    image_list = []
    for element in array:
        augmented_image = zoomin(image, element)
        image_list.append(augmented_image)
    return image_list


def zoomout(image, zoom_factor):
    height, width = image.shape[:2]
    new_width = int(width * zoom_factor)
    new_height = int(height * zoom_factor)

    # Compute the aspect ratio difference
    width_ratio = new_width / width
    height_ratio = new_height / height
    aspect_ratio = min(width_ratio, height_ratio)

    # Compute the new dimensions
    new_width = int(width * aspect_ratio)
    new_height = int(height * aspect_ratio)

    # Ensure that the new dimensions are not larger than the original image dimensions
    new_width = min(new_width, width)
    new_height = min(new_height, height)

    # Compute the black border dimensions
    border_width = width - new_width
    border_height = height - new_height

    # Create a black border around the zoomed-out image
    border_color = (0, 0, 0)  # Black color
    bordered_image = cv2.copyMakeBorder(image, border_height, border_height, border_width, border_width,
                                        cv2.BORDER_CONSTANT, value=border_color)
    zoomed_out_image = cv2.resize(bordered_image, (width, height))
    return zoomed_out_image

def augmentation_zoomout(image):
    array = [0.8,0.7]
    image_list = []
    for element in array:
        augmented_image = zoomout(image, element)
        image_list.append(augmented_image)
    return image_list

def augmentation_meta(image):
    # Adjust brightness and contrast of the image
    array = np.arange(0.5, 2.5, 0.5)
    image_list = []
    for element in array:
        brightness = element  # Brightness factor (0.0 to 1.0)
        contrast = element + 1  # Contrast factor (>1.0 for higher contrast)
        augmented_image = cv2.convertScaleAbs(image, alpha=contrast, beta=brightness)
        image_list.append(augmented_image)
    return image_list

# 8. Augment Data Process and Save to AUGMENTED_FOLDER

In [ ]:
# Function to augment and save augmented keypoints
def augmented_background(frame_aug, file_num, frame_number, action):
    seq_num = file_num
    augmented_images = (
        augmentation_rotation(frame_aug, 45) +
        augmentation_zoomin(frame_aug) +
        augmentation_zoomout(frame_aug) +
        augmentation_meta(frame_aug)
    )
    
    for index, aug_image in enumerate(augmented_images):
        image, results = mediapipe_detection(aug_image, holistic)
        keypoints = extract_keypoints(results)
        npy_path = os.path.join(AUGMENTED_DATA_PATH, action, str(seq_num), str(frame_number))
        if not os.path.exists(os.path.join(AUGMENTED_DATA_PATH, action, str(seq_num))):
            os.makedirs(os.path.join(AUGMENTED_DATA_PATH, action, str(seq_num)))
        print(npy_path)
        np.save(npy_path, keypoints)
        seq_num += 1

In [ ]:
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Loop through actions
    for action in actions:
        seq_num = 27  # Starting sequence number for augmented sequences
        # Loop through sequences
        for train_index, sequence in enumerate(train_sequences):
            # Loop through sequence length
            for frame_num in range(sequence_length):
                # [READ - IMPORTANT - SIDANG - v]
                # Construct the path for the original .jpg image corresponding to the X_train sequence
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                frame = cv2.imread(f'{npy_path}.jpg')  # Load the image from the path

                if frame is not None:
                    # Augment and save the augmented keypoints
                    augmented_background(frame, seq_num, frame_num, action)
            
            # Increment sequence number to avoid overlap
            seq_num += 12

# 9. Combining Train Sequences + Augmented Data

In [ ]:
label_map = {label:num for num, label in enumerate(actions)}

from tensorflow.keras.utils import to_categorical

sequences, labels = [], []
for action in actions:
    for sequence in range(end_augmented_sequence+1):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(AUGMENTED_DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

# ==
X_TRAIN_FINAL = np.array(sequences)
y_train_final = to_categorical(labels).astype(int)


# 10. Build and Train BIGRU Neural Network

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(Bidirectional(GRU(64, return_sequences=True, activation='relu', input_shape=(30,126))))
model.add(Bidirectional(GRU(128, return_sequences=False, activation='relu')))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [ ]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
# Split the remaining data to train and validation
x_train, x_val, y_train, y_val = train_test_split(X_TRAIN_FINAL, y_train_final, test_size=0.1, shuffle=True)

In [ ]:
history = model.fit(x_train, y_train, epochs=50, callbacks=[tb_callback], validation_data=(x_val, y_val), batch_size=32)

In [ ]:
model.summary()

In [ ]:
# Plot training & validation loss values
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation accuracy values
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# 11. Make Predictions

In [ ]:
res = model.predict(X_test)

In [ ]:
actions[np.argmax(y_test[0])]

# 12. Save Models

In [ ]:
model.save('BISINDO_30Sep2024_v6.h5')

# ================================
## CHECK TRAIN LOGS with TENSORBOARD

1. Open a CMD
2. CD to the Logs train folder (in my case: "C:\."YOUR NAME"\SKRIPSI-Sign Language\SCRIPT\2_TRAINING\Logs\train")
3. And Type below

tensorboard --logdir=.

===

# 13. Evaluation - Confusion Matrix

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score

In [ ]:
yhat = model.predict(X_test)

In [ ]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

In [ ]:
# define class labels
class_labels = ['Hai', 'Aku', 'Cinta', 'Kamu', 'Teman', 'Makan', 'Hari Ini', 'Bantu', 'Terima Kasih', 'Maaf']

# ytrue and yhat are the predicted and the actual labels
conf_matrix = confusion_matrix(ytrue, yhat)

accuracy = accuracy_score(ytrue, yhat)
precision = precision_score(ytrue, yhat, average='weighted')
recall = recall_score(ytrue, yhat, average='weighted')
f1 = f1_score(ytrue, yhat, average='weighted')

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.set(font_scale=1.2)
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False,
            xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

print("\nAccuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-Score:", f1)

# Calculate classification report
report = classification_report(ytrue, yhat, target_names=class_labels)
print("\nClassification Report:\n", report)

# 14. Real-Time Test Program

In [ ]:
model.load_weights('BISINDO_30Sep2024_v6.h5')

In [ ]:
from scipy import stats

colors = [(245,117,16), (117,245,16), (16,117,245), (120,35,140), (200,13,160), (15,20,200), (90,20,10), (40,50,30), (140,100, 10), (200,30,60)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        # MAC OS FRAME RECTANGLE
        
        # cv2.rectangle(image, start_point, end_point, color, thickness)
        # cv2.putText(image, text, org (coordinates), font, fontScale, color[, thickness[, lineType[, bottomLeftOrigin]]])
        
        # Pembesaran Frame Rectangle = WORKED but masih KECIL
        cv2.rectangle(output_frame, (0, 200+num*200), (int(prob*480), 300+num*200), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 260+num*200), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 5, cv2.LINE_AA)

        # Kasih Tambahan Angka Probabilitas 
        cv2.rectangle(output_frame, (500, 200+num*200), (650, 300+num*200), (0, 0, 0), 3)
        cv2.putText(output_frame, ' ' + str(round(prob*100, 1)) + '%', (500, 260+num*200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            # print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            # # Filter out the noise
            if np.argmax(np.bincount(predictions[-10:]))==np.argmax(res):
            # if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (2000, 150), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (30,100), 
                       cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 0), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('Hazlan BISINDO REAL-TIME TEST | WEBCAM', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

# 15. **[Bonus]** Step Perhitungan Arsitektur BiGRU (Simulation Testing)

In [ ]:
import torch
import torch.nn.functional as F

# Input: Single keypoint value 0.1
X = torch.tensor([[0.5]])

# Define the GRU network parameters:
hidden_size = 1  # GRU layer with only one unit

### 2. First BiGRU Layer
# Shared weights for update, reset, and new state gates (for simplicity)
W_u = torch.tensor([[0.5]])
W_r = torch.tensor([[0.5]])
W = torch.tensor([[0.5]])  # W for candidate hidden state
Q_u = torch.tensor([[0.2]])
Q_r = torch.tensor([[0.2]])
Q = torch.tensor([[0.2]])  # Q for candidate hidden state
b_u = torch.tensor([0.1])
b_r = torch.tensor([0.1])
b = torch.tensor([0.1])    # b for candidate hidden state

# Initialize the hidden state for the forward and backward cells of the first BiGRU layer
h_t_f1 = torch.zeros(1)  # Forward cell
h_t_b1 = torch.zeros(1)  # Backward cell

# GRU cell calculation for the first BiGRU layer - forward cell
u_f1 = torch.sigmoid(torch.matmul(X, W_u) + b_u + torch.matmul(h_t_f1, Q_u) + b_u)
r_f1 = torch.sigmoid(torch.matmul(X, W_r) + b_r + torch.matmul(h_t_f1, Q_r) + b_r)
h_tilde_f1 = torch.tanh(torch.matmul(X, W) + b + r_f1 * (torch.matmul(h_t_f1, Q) + b))
h_t_f1_new = u_f1 * h_t_f1 + (1 - u_f1) * h_tilde_f1

# GRU cell calculation for the first BiGRU layer - backward cell
u_b1 = torch.sigmoid(torch.matmul(X, W_u) + b_u + torch.matmul(h_t_b1, Q_u) + b_u)
r_b1 = torch.sigmoid(torch.matmul(X, W_r) + b_r + torch.matmul(h_t_b1, Q_r) + b_r)
h_tilde_b1 = torch.tanh(torch.matmul(X, W) + b + r_b1 * (torch.matmul(h_t_b1, Q) + b))
h_t_b1_new = u_b1 * h_t_b1 + (1 - u_b1) * h_tilde_b1

# Combine the forward and backward hidden states using element-wise multiplication
h_t_1 = h_t_f1_new * h_t_b1_new

# Apply ReLU activation after combining
h_t_1_relu = F.relu(h_t_1)

### 3. Second BiGRU Layer
# Initialize the hidden state for the forward and backward cells of the second BiGRU layer
h_t_f2 = torch.zeros(1)  # Forward cell
h_t_b2 = torch.zeros(1)  # Backward cell

# GRU cell calculation for the second BiGRU layer - forward cell
u_f2 = torch.sigmoid(torch.matmul(X, W_u) + b_u + torch.matmul(h_t_1_relu, Q_u) + b_u)
r_f2 = torch.sigmoid(torch.matmul(X, W_r) + b_r + torch.matmul(h_t_1_relu, Q_r) + b_r)
h_tilde_f2 = torch.tanh(torch.matmul(X, W) + b + r_f2 * (torch.matmul(h_t_1_relu, Q) + b))
h_t_f2_new = u_f2 * h_t_1_relu + (1 - u_f2) * h_tilde_f2

# GRU cell calculation for the second BiGRU layer - backward cell
u_b2 = torch.sigmoid(torch.matmul(X, W_u) + b_u + torch.matmul(h_t_1_relu, Q_u) + b_u)
r_b2 = torch.sigmoid(torch.matmul(X, W_r) + b_r + torch.matmul(h_t_1_relu, Q_r) + b_r)
h_tilde_b2 = torch.tanh(torch.matmul(X, W) + b + r_b2 * (torch.matmul(h_t_1_relu, Q) + b))
h_t_b2_new = u_b2 * h_t_1_relu + (1 - u_b2) * h_tilde_b2

# Combine the forward and backward hidden states using element-wise multiplication
h_t_2 = h_t_f2_new * h_t_b2_new

# Apply ReLU activation after combining
h_t_2_relu = F.relu(h_t_2)

### 4. Dense Layer
# Dense layer weights and bias
W_dense = torch.tensor([[0.7]])
b_dense = torch.tensor([0.2])

# Dense layer output
y_linear = torch.matmul(h_t_2_relu, W_dense.T) + b_dense

# Apply ReLU activation after the dense layer
h_dense_relu = F.relu(y_linear)

### 5. Output Layer
# Softmax activation for output layer
y_linear_2 = torch.matmul(h_dense_relu, W_dense.T) + b_dense

y_pred = F.softmax(y_linear_2, dim=0)

# Print detailed calculations
print("\n=== First BiGRU Layer - Forward Cell ===")
print(f"Update Gate (u_f1): {u_f1.item()}")
print(f"Reset Gate (r_f1): {r_f1.item()}")
print(f"Candidate Hidden State (h_tilde_f1): {h_tilde_f1.item()}")
print(f"New Hidden State (h_t_f1_new): {h_t_f1_new.item()}")

print("\n=== First BiGRU Layer - Backward Cell ===")
print(f"Update Gate (u_b1): {u_b1.item()}")
print(f"Reset Gate (r_b1): {r_b1.item()}")
print(f"Candidate Hidden State (h_tilde_b1): {h_tilde_b1.item()}")
print(f"New Hidden State (h_t_b1_new): {h_t_b1_new.item()}")

print(f"\n1st First Cell ReLU: {h_t_1_relu.item()}")

print("\n=== Second BiGRU Layer - Forward Cell ===")
print(f"Update Gate (u_f2): {u_f2.item()}")
print(f"Reset Gate (r_f2): {r_f2.item()}")
print(f"Candidate Hidden State (h_tilde_f2): {h_tilde_f2.item()}")
print(f"New Hidden State (h_t_f2_new): {h_t_f2_new.item()}")

print("\n=== Second BiGRU Layer - Backward Cell ===")
print(f"Update Gate (u_b2): {u_b2.item()}")
print(f"Reset Gate (r_b2): {r_b2.item()}")
print(f"Candidate Hidden State (h_tilde_b2): {h_tilde_b2.item()}")
print(f"New Hidden State (h_t_b2_new): {h_t_b2_new.item()}")

print(f"\n 2nd Second Cell ReLU: {h_t_2_relu.item()}")

print("\n=== First Dense Layer ===")
print(f"Linear Output (y_linear): {y_linear.item()}")

print("\n=== Second Dense Layer (Output) ===")
print(f"2nd Linear Output (y_linear_2): {y_linear_2.item()}")
print(f"Softmax Output (y_pred): {y_pred.item()}")